In [28]:
import output as oup 
import numpy as np

In [42]:
cur = oup.cursor('../cyclus-files/oat/cooling-time/ct1.sqlite')

In [5]:
hlw_waste = cur.execute('SELECT sum(quantity) FROM transactions '
                         'INNER JOIN `resources`'
                         'ON transactions.resourceid = resources.resourceid '
                         'WHERE Commodity = "lwrreprocessingwaste" '
                         'OR Commodity="frreprocessingwaste" OR Commodity="moxreprocessingwaste" ').fetchall()

In [6]:
hlw_waste[0][0]

12596672.199454095

In [7]:
depleted_u = cur.execute('SELECT sum(quantity) FROM transactions '
                         'INNER JOIN `resources`'
                         'ON transactions.resourceid = resources.resourceid '
                         'WHERE Commodity = "enrichmentwaste" ').fetchall()

In [8]:
depleted_u[0][0]

780762578.8995372

In [9]:
inventory = oup.stockpiles(cur,'lwrstorage',False)
max(inventory['lwrstorage'])

SELECT timecreated, quantity, qualid FROM resources INNER JOIN agentstateinventories ON agentstateinventories.resourceid = resources.resourceid WHERE (agentid = "131")


ValueError: max() arg is an empty sequence

In [10]:
lwrstorage_id = oup.agent_ids(cur,'lwrstorage')

In [11]:
pu_lwrstorage = cur.execute('SELECT timecreated, sum(quantity*massfrac) '
                            'FROM resources INNER JOIN `agentstateinventories` '
                            'ON agentstateinventories.resourceid = resources.resourceid '
                            'left join `compositions`'
                            'on resources.QualId = Compositions.QualId WHERE '
                            'agentid = "131" and nucid like "94%"'
                            'group by timecreated').fetchall()

In [12]:
init_year, init_month, duration, timestep = oup.simulation_timesteps(cur)

In [13]:
oup.timeseries(pu_lwrstorage,duration,False)

[]

In [44]:
# The correct Pu in a facility 
stuffin = 'lwrout'
stuffout = 'lwrstorageout'
pu_in = cur.execute('SELECT time, sum(quantity*massfrac) ' 
'FROM transactions inner join resources '
'on Transactions.resourceid = resources.resourceid '
'left join compositions '
'on resources.QualId = Compositions.QualId '
'where commodity = "lwrout" and nucid like "94%" '
'group by time' ).fetchall()
pu_in_list = oup.timeseries(pu_in,duration,False)
pu_out = cur.execute('SELECT time, sum(quantity*massfrac) ' 
'FROM transactions inner join resources '
'on Transactions.resourceid = resources.resourceid '
'left join compositions '
'on resources.QualId = Compositions.QualId '
'where commodity = "lwrstorageout" and nucid like "94%" '
'group by time' ).fetchall()
pu_out_list = oup.timeseries(pu_out,duration,False)

In [47]:
N = len(pu_out_list)
pu_inventory = np.zeros(N)
pu_inventory[0] = pu_in_list[0]
for x in range(N): 
    pu_inventory[x] = pu_inventory[x-1] + pu_in_list[x] - pu_out_list[x]

In [48]:
pu_inventory

array([    0.        ,     0.        ,     0.        , ...,
       12567.04295016, 12567.04295016, 12567.04295016])

In [33]:
spent_fuel = cur.execute('SELECT time, sum(quantity) FROM transactions '
                         'INNER JOIN resources '
                         'ON transactions.resourceid = resources.resourceid '
                         'WHERE Commodity = "lwrreprocessingwaste" '
                         'GROUP BY time ').fetchall()

In [ ]:
spent_fuel[1][0]

In [ ]:
SELECT timecreated, quantity, qualid 
FROM resources INNER JOIN agentstateinventories 
ON agentstateinventories.resourceid = resources.resourceid 
WHERE (agentid = "230")